In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [18]:
df = pd.read_csv('/mnt/chicm/open-images-vrd/vrd/lb25388/sub_tuple_0825_cdiff.csv')
df.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.500165 /m/04yx4 0.3756532 0.236988 0.7501363...
1,763c72626bf24534,0.77768 /m/04yx4 0.720626 0.5248438 0.950433 0...
2,7540c162180973b1,0.114314 /m/04yx4 0.9498244 0.3313462 0.999023...
3,c4e522dce52cb02f,0.702323 /m/04yx4 0.8541945 0.3647041 0.972777...
4,cafcd9496c808085,0.178867 /m/01_5g 0.7968998 0.0202855 0.997210...


In [2]:
df = pd.read_csv('/mnt/chicm/open-images-vrd/vrd/sub0930/sub_all_0930.csv')
df.head()

,ImageId,PredictionString
0,00000b4dcff7f799,0.320416 /m/04yx4 0.7528 0.3605 0.8595 0.8582 ...
1,00001a21632de752,0.500611 /m/04yx4 0.6143 0.4868 0.7604 0.7347 ...
2,0000d67245642c5f,0.012464 /m/04ctx 0.2441 0.287 0.6494 0.896 /m...
3,0001244aa8ed3099,0.011098 /m/04dr76w 0.5264 0.4861 0.5508 0.595...
4,000172d1dd1adce0,0.005296 /m/04ctx 0.3545 0.0469 0.8018 0.7683 ...


In [3]:
DATA_DIR = '/mnt/chicm/data/open-images/relation'
df_test = pd.read_csv(os.path.join(DATA_DIR, 'VRD_sample_submission.csv'))
df_test.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...
1,763c72626bf24534,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...
2,7540c162180973b1,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...
3,c4e522dce52cb02f,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...
4,cafcd9496c808085,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...


In [4]:
df = df.set_index('ImageId')
df = df.reindex(index=df_test['ImageId'])
df = df.reset_index()
df.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.409728 /m/04yx4 0.3734 0.2024 0.7435 0.987 /...
1,763c72626bf24534,0.532883 /m/04yx4 0.7264 0.5176 0.9514 0.8937 ...
2,7540c162180973b1,0.348482 /m/04yx4 0.577 0.1447 0.8297 0.7084 /...
3,c4e522dce52cb02f,0.59702 /m/04yx4 0.8593 0.3911 0.983 0.8919 /m...
4,cafcd9496c808085,0.193685 /m/01_5g 0.7897 0.0374 0.9893 0.2174 ...


In [5]:
selected_rel = ['plays', 'holds', 'on', 'wears', 'interacts_with']
#selected_rel = 'on'
def get_filtered_pred_string(pred_str):
    if len(pred_str) < 1:
        return ''
    dets = []
    det = []
    for i, e in enumerate(pred_str.split(' ')):
        if i % 12 == 0:
            det = []
        det.append(e)
        if (i+1) % 12 == 0 and det[11] not in selected_rel:
            dets.append(' '.join(det))
                
    return ' '.join(dets)

In [6]:
df.rename(columns={'PredictionString': 'PredictionStringOld'}, inplace=True)
df.head()

,ImageId,PredictionStringOld
0,21135d5ed0e7a0b7,0.409728 /m/04yx4 0.3734 0.2024 0.7435 0.987 /...
1,763c72626bf24534,0.532883 /m/04yx4 0.7264 0.5176 0.9514 0.8937 ...
2,7540c162180973b1,0.348482 /m/04yx4 0.577 0.1447 0.8297 0.7084 /...
3,c4e522dce52cb02f,0.59702 /m/04yx4 0.8593 0.3911 0.983 0.8919 /m...
4,cafcd9496c808085,0.193685 /m/01_5g 0.7897 0.0374 0.9893 0.2174 ...


In [7]:
df.PredictionStringOld = df.PredictionStringOld.fillna('')

In [8]:
df['PredictionString'] = df.PredictionStringOld.map(lambda x: get_filtered_pred_string(x))
df.head()

,ImageId,PredictionStringOld,PredictionString
0,21135d5ed0e7a0b7,0.409728 /m/04yx4 0.3734 0.2024 0.7435 0.987 /...,0.219176 /m/04yx4 0.3734 0.2024 0.7435 0.987 /...
1,763c72626bf24534,0.532883 /m/04yx4 0.7264 0.5176 0.9514 0.8937 ...,0.532883 /m/04yx4 0.7264 0.5176 0.9514 0.8937 ...
2,7540c162180973b1,0.348482 /m/04yx4 0.577 0.1447 0.8297 0.7084 /...,0.173548 /m/04yx4 0.0295 0.1986 0.0747 0.3123 ...
3,c4e522dce52cb02f,0.59702 /m/04yx4 0.8593 0.3911 0.983 0.8919 /m...,0.59702 /m/04yx4 0.8593 0.3911 0.983 0.8919 /m...
4,cafcd9496c808085,0.193685 /m/01_5g 0.7897 0.0374 0.9893 0.2174 ...,0.084175 /m/01mzpv 0.8351 0.0081 0.9989 0.3012...


In [9]:
df.to_csv('/mnt/chicm/open-images-vrd/vrd/sub0930/sub_all_0930_excludes_holds_plays_on_wears_interacts_with.csv', columns=['ImageId', 'PredictionString'], index=False)

In [84]:
!kaggle competitions submit -c open-images-2019-visual-relationship -f /mnt/chicm/open-images-vrd/vrd/lb25388/sub_tuple_0825_cdiff_on.csv -m 'on'

100%|████████████████████████████████████████| 468M/468M [00:23<00:00, 20.6MB/s]
Successfully submitted to Open Images 2019 - Visual Relationship